In [ ]:
%pip install snowflake-snowpark-python
%pip install boto3

In [1]:
import json
import os
from snowflake.snowpark import Session
from snowflake.snowpark.functions import sproc
import snowflake.snowpark
from snowflake.snowpark.types import IntegerType
import boto3

In [2]:
import logging
import sys

In [3]:
with open("./config.json", "r") as f:
    connection_parameters = json.loads(f.read())
test_session = Session.builder.configs(connection_parameters).create()

In [4]:
bucket = "airbyte.alex"
connector = "source-faker"
s3_paginator = boto3.client('s3').get_paginator('list_objects_v2')

def keys(bucket_name, prefix='/', delimiter='/', start_after=''):
    prefix = prefix[1:] if prefix.startswith(delimiter) else prefix
    start_after = (start_after or prefix) if prefix.endswith(delimiter) else start_after
    for page in s3_paginator.paginate(Bucket=bucket_name, Prefix=prefix, StartAfter=start_after):
        for content in page.get('Contents', ()):
            yield content['Key']
            
keys = list(keys(bucket, prefix=f"{connector}"))

# Compute the list of files to import in the proc
# For now we're just loading everything
files_to_load = [f"@mystage/{k}" for k in keys if "pendulum" not in k]

In [5]:
import snowflake.snowpark
from snowflake.snowpark.functions import sproc
from snowflake.snowpark.types import IntegerType

# Create a stage to store the code
create_stage_result = test_session.sql(f"create or REPLACE stage mystage url = 's3://{bucket}'").collect()

# pendulum has to be installed as a package for reasons unknown...
@sproc(packages=['snowflake-snowpark-python', 'pendulum'], imports=files_to_load, name="myproc2", replace=True)
def compute(session: snowflake.snowpark.Session, to_table: str) -> str:
    from faker import Faker
    from airbyte_cdk.models import ConfiguredAirbyteCatalog

    from source_faker import SourceFaker
    logger = logging.getLogger("logger")
    source = SourceFaker()
    config = {"count": 5, "seed": -1, "records_per_sync": 10, "records_per_slice": 3}
    # Discover doesn't actually work because it can't load the json files from the zip file
    catalog = source.discover(logger, config)
    configured_catalog = ConfiguredAirbyteCatalog(
        streams=[ConfiguredAirbyteStream(stream=s, sync_mode=SyncMode.full_refresh, destination_sync_mode=DestinationSyncMode.append) for s
                 in catalog.streams])
    data = list(source.read(logger, config, configured_catalog, {}))
    session.create_dataframe(data).write.mode('append').save_as_table(to_table)
    return "SUCCESS"

In [6]:
print(test_session.sql("select count(*) from public.test_table3;").collect())
print(test_session.call("myproc2", "public.test_table3"))
print(test_session.sql("select * from public.test_table3;").collect())

[Row(COUNT(*)=36)]


Failed to execute query [queryID: None] CALL myproc2('public.test_table3')
100357 (P0000): Python Interpreter Error:
Traceback (most recent call last):
  File "_udf_code.py", line 7, in compute
  File "/var/folders/xx/m2btk9zx2436dd6nldyjtggh0000gn/T/ipykernel_70020/63871477.py", line 16, in compute
  File "/home/udf/11822039421/source_faker.zip/source_faker/source.py", line 70, in discover
    catalog = read_json(spec_path)
  File "/home/udf/11822039421/source_faker.zip/source_faker/source.py", line 254, in read_json
    with open(filepath, "r") as f:
NotADirectoryError: [Errno 20] Not a directory: '/home/udf/11822039421/source_faker.zip/source_faker/users_catalog.json'
 in function MYPROC2 with handler compute


SnowparkSQLException: (1304): 100357 (P0000): Python Interpreter Error:
Traceback (most recent call last):
  File "_udf_code.py", line 7, in compute
  File "/var/folders/xx/m2btk9zx2436dd6nldyjtggh0000gn/T/ipykernel_70020/63871477.py", line 16, in compute
  File "/home/udf/11822039421/source_faker.zip/source_faker/source.py", line 70, in discover
    catalog = read_json(spec_path)
  File "/home/udf/11822039421/source_faker.zip/source_faker/source.py", line 254, in read_json
    with open(filepath, "r") as f:
NotADirectoryError: [Errno 20] Not a directory: '/home/udf/11822039421/source_faker.zip/source_faker/users_catalog.json'
 in function MYPROC2 with handler compute

In [ ]:
test_session.close()